In [99]:
import pandas as pd

In [103]:
df_transformer = pd.read_csv('/home/sanjana/factual_evaluation_source_based/datasets/sota_annotations/dialogue_aggrefact.csv')
df_llms_annot1 = pd.read_csv('/home/sanjana/factual_evaluation_source_based/annotations/sample/sample_annotations_sanjana.csv')
df_llms_annot1 = df_llms_annot1.sort_values('uuid').drop_duplicates(['docid'], keep='last')

df_llms_annot1.head()

,Unnamed: 0,uuid,user_id,docid,model,nonfactual_spans,evidence,error_type,summary,dialogue
34,34,07a976d3-9c2d-4ca3-a6c5-d9f868c2420b,sanjana,test_27,GPT4,NaN,NaN,NaN,James and Muriel Douglas meet for the first ti...,"#Person1#: Hello, are you Muriel Douglas?\r\n#..."
5,5,125445b3-5923-484d-bb84-f155cf165f89,sanjana,test_133,GPT4,enjoyed nice weather in March,#Person1#: : I suppose we should not complain....,Intrinsic_Error,Person1 and Person2 discussed the heavy storm ...,"#Person1#: It was a heavy storm last night, wa..."
46,46,1464fef3-a336-4866-8072-fe5e3fa9352d,sanjana,test_306,GPT4,NaN,NaN,NaN,Person1 accuses Person2 of not paying attentio...,"#Person1#: Like a cat on hot bricks, as you mi..."
60,60,1e24ddb1-9830-4bc9-99b6-888f6091f485,sanjana,test_350,GPT4,NaN,NaN,NaN,Person1 inquired about their delayed flight CA...,"#Person1#: Excuse me, Miss.\r\n#Person2#: What..."
47,47,1fa180c8-0b94-47b0-807a-495ca6c300a6,sanjana,test_315,GPT4,NaN,NaN,NaN,Person1 asked Person2 about their language ski...,#Person1#: How many languages can you speak?\r...


In [104]:
df_llms_annot2 = pd.read_csv('/home/sanjana/factual_evaluation_source_based/annotations/sample/sample_annotations_elisa.csv')
df_llms_annot2 = df_llms_annot2.sort_values('uuid').drop_duplicates(['docid'], keep='last')


df_llms_annot2.head()

,Unnamed: 0,uuid,user_id,docid,model,nonfactual_spans,evidence,error_type,summary,dialogue
0,0,6,elisa,test_12,GPT4,Person 1 (Steven) noticed that Pe,What happene,Intrinsic_Error,Person 1 (Steven) noticed that Person 2 (Tony)...,"#Person1#: Hi, Tony. You look unhappy. What's ..."
1,1,7,elisa,test_120,GPT4,NaN,NaN,NaN,"Person1 checks if Person2 has fed the puppies,...",#Person1#: Have you given the puppies food yet...
2,2,8,elisa,test_123,GPT4,Person1 inquires about the me,NaN,Intrinsic_Error,Person1 inquires about the method to contact P...,#Person1#: How can we get in touch with you?\r...
4,4,10,elisa,test_127,GPT4,"s, and earned a weekl",NaN,Intrinsic_Error,Person1 asked Mr. Smith (Person2) about his ed...,#Person1#: Could you tell me what university y...
5,5,11,elisa,test_132,GPT4,"going, caring, and non-",NaN,Intrinsic_Error,Person1 and Person2 discuss their idea of the ...,#Person1#: Hey Rocky! You've been sitting arou...


In [105]:
len(df_llms_annot1), len(df_llms_annot2)

(74, 74)

In [127]:
import math 
agreement_count = 0
agreement_error_types = 0 
total_agreed_error = 0
total_disagreed_error = 0
diagreed_docs = []
# len(set(df_llms_annot1['docid'].values)), len(set(df_llms_annot2['docid'].values))
for docid in list(set(df_llms_annot2['docid'].values)):
    # print(docid, len(ann2_row))
    ann1_row = df_llms_annot1[df_llms_annot1['docid'] == docid]
    ann2_row = df_llms_annot2[df_llms_annot2['docid'] == docid]

    error_type_ann1 = ann1_row['error_type'].values[-1]
    error_type_ann1 = '' if type(error_type_ann1) != str else error_type_ann1
    error_type_ann2 = ann2_row['error_type'].values[-1]
    error_type_ann2 = '' if type(error_type_ann2) != str else error_type_ann2
    if error_type_ann1 == error_type_ann2:
        agreement_count += 1

    if error_type_ann1 != '' and error_type_ann2 != '':
        total_agreed_error += 1
        if error_type_ann1 == error_type_ann2:
            agreement_error_types += 1

    if (not(error_type_ann1 != '' and error_type_ann2 != '')) and (error_type_ann1 != '' or error_type_ann2 != '') :
        diagreed_docs += [docid]
        total_disagreed_error += 1
# agreement_count/len(list(set(df_llms_annot2['docid'].values))), agreement_error_types/total_agreed_error

total_disagreed_error/len(list(set(df_llms_annot2['docid'].values)))

0.25675675675675674

In [140]:
df_llms_annot2_dis = df_llms_annot2[df_llms_annot2['docid'].isin(diagreed_docs)]

In [141]:
df_llms_annot1_dis = df_llms_annot1[df_llms_annot1['docid'].isin(diagreed_docs)]

In [142]:
df_llms_annot1_dis = df_llms_annot1_dis.sort_values(by=['docid'], ascending=True)

In [143]:
df_llms_annot2_dis = df_llms_annot2_dis.sort_values(by=['docid'], ascending=True)

In [144]:
df_llms_annot2_dis['user_id2'] = df_llms_annot1_dis['user_id'].values
df_llms_annot2_dis['nonfactual_spans2'] = df_llms_annot1_dis['nonfactual_spans'].values
df_llms_annot2_dis['evidence2'] = df_llms_annot1_dis['evidence'].values
df_llms_annot2_dis['error_type2'] = df_llms_annot1_dis['error_type'].values


In [145]:
df_llms_annot2_dis

,Unnamed: 0,uuid,user_id,docid,model,nonfactual_spans,evidence,error_type,summary,dialogue,user_id2,nonfactual_spans2,evidence2,error_type2
0,0,6,elisa,test_12,GPT4,Person 1 (Steven) noticed that Pe,What happene,Intrinsic_Error,Person 1 (Steven) noticed that Person 2 (Tony)...,"#Person1#: Hi, Tony. You look unhappy. What's ...",sanjana,NaN,NaN,NaN
2,2,8,elisa,test_123,GPT4,Person1 inquires about the me,NaN,Intrinsic_Error,Person1 inquires about the method to contact P...,#Person1#: How can we get in touch with you?\r...,sanjana,NaN,NaN,NaN
4,4,10,elisa,test_127,GPT4,"s, and earned a weekl",NaN,Intrinsic_Error,Person1 asked Mr. Smith (Person2) about his ed...,#Person1#: Could you tell me what university y...,sanjana,NaN,NaN,NaN
5,5,11,elisa,test_132,GPT4,"going, caring, and non-",NaN,Intrinsic_Error,Person1 and Person2 discuss their idea of the ...,#Person1#: Hey Rocky! You've been sitting arou...,sanjana,NaN,NaN,NaN
6,6,12,elisa,test_133,GPT4,NaN,NaN,NaN,Person1 and Person2 discussed the heavy storm ...,"#Person1#: It was a heavy storm last night, wa...",sanjana,enjoyed nice weather in March,#Person1#: : I suppose we should not complain....,Intrinsic_Error
8,8,14,elisa,test_140,GPT4,Person3 that Person1 is p,NaN,Intrinsic_Error,"Person1 forgives Person2 for not calling, who ...","#Person1#: Honey, of course I forgive you! I l...",sanjana,NaN,NaN,NaN
9,9,15,elisa,test_147,GPT4,ating whether dr,NaN,Intrinsic_Error,Person1 and Person2 are arguing about whose fa...,#Person1#: This is all your fault. This wouldn...,sanjana,NaN,NaN,NaN
16,16,22,elisa,test_161,GPT4,NaN,NaN,NaN,"Andy approached Fanny, who sounded tired due t...","#Person1#: Hi, Fanny.\r\n#Person2#: Hi, Andy.\...",sanjana,They both agreed that if they aren't accepted ...,#Person1#: : Me too. But I tell myself that if...,Intrinsic_Error
20,20,26,elisa,test_191,GPT4,scheduled,NaN,Intrinsic_Error,"Person1 inquired about their test results, lea...","#Person1#: Hi, I was wondering if I could get ...",sanjana,NaN,NaN,NaN
22,22,28,elisa,test_194,GPT4,it seems the battery is dead,NaN,Intrinsic_Error,Person1 realizes they have misplaced their cel...,"#Person1#: Linda, I can't find my cellphone an...",sanjana,NaN,NaN,NaN


In [146]:
df_llms_annot2_dis.to_csv('disagreement_errortypes.csv')

In [ ]:
df_transformer
error_key = 'w/ Error'
error_key = 'error_type'
error_vals = [1]
# df_errors = df[df[error_key].isin(error_val)]
# df = {'
import seaborn as sns
import matplotlib.pyplot as plt
# set a grey background (use sns.set_theme() if seaborn version 0.11.0 or above) 
sns.set(style="darkgrid")
sns.histplot(data=df_llms, x=error_key)
plt.show()

In [ ]:
df_llms